In [2]:
import csv

# Définition des classes

In [3]:
livres = []
langues = {}
auteurs = {}
editeurs = {}
titres = []
categoriesujet = {}
sujets = {}
typepublication = {}
typedocument = {}

liste_codes = { 
                "16.32":"Poésie", 
                "16.33":"Théatre",
                "16.34":"Roman",
                "16.35":"Contes/Nouvelles",
                "16.38":"Jeunesse",
                "16.41":"Bandes dessinées"}
    

def extraireTitre(chaine):
    return  chaine.split(' / ')[0].split(' : ')[0]
                             

class Livre:
    def __init__(self, titre = "***NO TITLE***"):
        self.id_depot_legal = ""
        self.titre = titre
        self.type = ""
        self.langue = ""
        self.langue_originale = ""
        self.annee = ""
        self.auteurs = []
        self.isbn = []
        self.editeur = None
        
    def show(self):
        print('---', self.id_depot_legal, self.titre, '---')
        print('   ',self.type, self.langue, end=" ")
        if self.langue_originale != self.langue:
            print("({})".format(self.langue_originale), end=" ")
        print(self.annee)
        for a in self.auteurs:
            print('   ', a.prenom, a.nom, end = ' : ')
        print()
        print(self.isbn)

class Auteur():
    def __init__(self,id,nom = ""):
        self.id = id
        nom_decompose = nom.split(',')
        self.nom = nom_decompose[0].strip()
        self.entites = {}
        if len(nom_decompose) > 1:
            self.prenom = nom_decompose[1].strip()
        else:
            self.prenom = ""
            
    def nom_complet(self):
        return self.prenom + ' ' + self.nom
  
        
class Editeur():
    def __init__(self,nom = ""):
        self.nom = nom
        self.code = ""
        self.categorie = ""
    

# Extraction des données du dépot légal

In [5]:
auteur_id = 0

with open('../Data/DepotLegal/depotlegal20171231.csv') as fichier_depot_legal:
    depot_reader = csv.DictReader(fichier_depot_legal)
    for data in depot_reader:
        if data['SUJET'] in sujets:
            sujets[data['SUJET']] += 1
        else:
            sujets[data['SUJET']] = 1

        if data['TYPEPUBLICATION'] in typepublication:
            typepublication[data['TYPEPUBLICATION']] += 1
        else:
            typepublication[data['TYPEPUBLICATION']] = 1
            
        if data['TYPE_DOCUMENT'] in typedocument:
            typedocument[data['TYPE_DOCUMENT']] += 1
        else:
            typedocument[data['TYPE_DOCUMENT']] = 1
  
        if data['CATEGORIE_SUJET'] in categoriesujet:
            categoriesujet[data['CATEGORIE_SUJET']] += 1
        else:
            categoriesujet[data['CATEGORIE_SUJET']] = 1
 
        if data['LANGUE_PUBLICATION'] in langues:
            langues[data['LANGUE_PUBLICATION']] += 1
        else:
            langues[data['LANGUE_PUBLICATION']] = 1            
            
        code_sujet = data['SUJET'][:5]
        if code_sujet in liste_codes:
            livre = Livre(extraireTitre(data["TITRE_PUBLICATION"]))
            titres.append(data["TITRE_PUBLICATION"])
            livre.type = liste_codes[code_sujet]
            livre.annee = data['ANNEE_PUBLICATION']
            livre.langue = data['LANGUE_PUBLICATION']
            livre.id_depot_legal = data['ID_DEPOT']
            if data['LISTE_ISBN_NETTOYE'] != '':
                livre.isbn = data['LISTE_ISBN_NETTOYE'].split(';')
            if data['LANGUE_ORIGINALE'] != "":
                livre.langue_originale = data['LANGUE_ORIGINALE']
            else:
                livre.langue_originale = livre.langue
            for a in data["LISTE_AUTEUR"].split(';'):
                if a not in auteurs:
                    auteur_id += 1
                    auteurs[a] = Auteur(auteur_id,a)
                livre.auteurs.append(auteurs[a])
            
            editeur = Editeur(data['NOM_EDITEUR'])
            editeur.code = data['CODE_EDITEUR']
            editeur.categorie = data['CATEGORIE_EDITEUR']
            
            if data['CODE_EDITEUR'] not in editeurs:
                editeurs[data['CODE_EDITEUR']] = editeur
                
            livre.editeur = editeur
            
            livres.append(livre)

In [9]:
print(len(auteurs), 'auteurs')
print(len(livres), 'livres')
print(len(editeurs))

14420 auteurs
56962 livres
4505


# Création du graphe RDF

In [10]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD


BANQ = Namespace("http://banq.qc.ca/")
SCHEMA = Namespace("https://schema.org/")
DCTERMS = Namespace("http://purl.org/dc/terms/")
DBO = Namespace("http://dbpedia.org/ontology/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")
WD = Namespace("http://www.wikidata.org/entity/")



In [11]:
grapheDepotLegal = Graph()


def creerUriAuteur(auteur):
    return BANQ['auteur/' + str(auteur.id)]

def creerUriEditeur(editeur):
    return BANQ['editeur/' + str(editeur.code)]


for a in auteurs.values():
    grapheDepotLegal.add((creerUriAuteur(a), RDF.type, DBO['Writer']))
    grapheDepotLegal.add((creerUriAuteur(a), SCHEMA.givenName, Literal(a.prenom)))
    grapheDepotLegal.add((creerUriAuteur(a), SCHEMA.familyName, Literal(a.nom)))

for e in editeurs:
    grapheDepotLegal.add((creerUriEditeur(editeurs[e]), RDF.type, DBO['Publisher']))   
    grapheDepotLegal.add((creerUriEditeur(editeurs[e]), SCHEMA.name, Literal(editeurs[e].nom)))
    
    
for l in livres:
    livre_uri = BANQ['livre/' + str(l.id_depot_legal)]
    grapheDepotLegal.add((livre_uri,SCHEMA.name,Literal(l.titre)))
    grapheDepotLegal.add((livre_uri,RDF.type,DBO['Book']))

    for a in l.auteurs:
        grapheDepotLegal.add((livre_uri,SCHEMA.author,creerUriAuteur(a)))
    for i in l.isbn:
        grapheDepotLegal.add((livre_uri,SCHEMA.isbn,Literal(i)))
    
    grapheDepotLegal.add((livre_uri,SCHEMA.inLanguage,Literal(l.langue)))
    grapheDepotLegal.add((livre_uri,BANQ.type,Literal(l.type)))
    grapheDepotLegal.add((livre_uri,SCHEMA.datePublished,Literal(l.annee)))

    grapheDepotLegal.add((livre_uri,SCHEMA.publisher,creerUriEditeur(l.editeur)))



56962 livres
14420 auteurs
4505 éditeurs
533726 triplets dans le graphe


In [16]:
print(len(grapheDepotLegal),'triplets dans le graphe') 


533726 triplets dans le graphe


In [17]:
# pour tester

for s,p,o in grapheDepotLegal.triples( (BANQ['auteur/' + '3473'], None, None) ):
   print(p,o)

livres[2378].show()

https://schema.org/givenName Jean-Marie
https://schema.org/familyName Gignac
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/Writer
--- 24401 "Le temps, l'espace et les autres demains" ---
    Poésie Français 1993
    Renée Vaillancourt-Lauzière : 
['2893630057']


In [18]:
fichier_depot_legal = open("../Graphes/grapheDepotLegal.rdf","wb")
grapheDepotLegal.serialize(fichier_depot_legal) 
fichier_depot_legal.close()


# À faire

* Dans le cas de traductions (information indiquée dans le titre), il faudrait retrouver l'oeuvre originale
* Il faut réfléchir à la catégorisation des livres (taxonomie)
* Liens avec Wikidata, DBpedia et autres...
* Plusieurs parutions des mêmes oeuvres
* Traitement des titres nécessaire; ils contiennent souvent de l'information supplémentaire (format non standardisé):
    * Livre en gros caractères
    * Traducteur
    * Auteur
    * Illustrateur
    * Catégorie
    * Témoignages et notes
    * Adaptation
    * Autres